In [1]:
%load_ext autoreload
%autoreload 2
    
import fiona
from pathlib import Path
import metapack as mp
import geopandas as gpd
import pandas as pd
from auto_tqdm import tqdm 
import rowgenerators as rg

doc = mp.jupyter.open_source_package()
doc.set_sys_path()
from  pylib import * 


import logging
logging.basicConfig()

from pylib import lines_logger, points_logger
lines_logger.setLevel(logging.DEBUG)
points_logger.setLevel(logging.DEBUG)

import appnope

pkg_root = Path(doc.path).parent
pkg = mp.open_package(pkg_root)
cache = open_cache(pkg)
pkg

# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-1.2.1` Last Update: 2021-02-15T17:38:34

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [point_tags](data/point_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash
* ** [block_geo](data/block_geo.csv)**. Geographic shapes for blocs in block_osm
* ** [block_osm](data/block_osm.csv)**. Points from geohash_tags allocated to 2020 census blocks
* ** [utm_map](data/utm_map.csv)**. Map from census blocks to UTM zones
* ** [cbsa_map](data/cbsa_map.csv)**. Map from census blocks to CBSA Metros
* ** [bus_densities](notebooks/PointDensity.ipynb#bus_densities)**. High-density business goehashes

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><ul>

In [7]:
from pylib.clusters import * 

cache.get(str(cache.list('clusters/buffered')[4]))

,cluster_n,cluster_type,geometry
0,0,ent,"POLYGON ((-81.60701 41.01240, -81.60702 41.012..."
1,1,ent,"POLYGON ((-81.63676 41.02561, -81.63674 41.025..."
2,2,ent,"POLYGON ((-81.63606 41.03610, -81.63963 41.036..."
3,3,ent,"POLYGON ((-81.60630 41.03474, -81.60630 41.034..."
4,4,ent,"POLYGON ((-81.60653 41.04074, -81.61010 41.040..."
...,...,...,...
511,511,casual,"POLYGON ((-81.35041 41.31539, -81.35400 41.315..."
512,512,casual,"POLYGON ((-81.35116 41.32084, -81.35116 41.320..."
513,513,casual,"POLYGON ((-81.22060 41.30921, -81.22062 41.309..."
514,514,casual,"POLYGON ((-81.17984 41.24359, -81.17984 41.243..."


In [11]:
for p in tqdm(cache.list('clusters/buffered')):
    df = cache.get(p)
    if 'cbsa' not in df.columns:
        df['cbsa'] = str(p.stem)
        cache.put(p,df)

  0%|          | 0/811 [00:00<?, ?it/s]

In [12]:
for p in tqdm(cache.list('clusters/points')):
    df = cache.get(p)
    if 'cbsa' not in df.columns:
        df['cbsa'] = str(p.stem)
        cache.put(p,df)

  0%|          | 0/822 [00:00<?, ?it/s]

In [13]:
df

,geoid,geohash,geometry,entertain,casual,shop,active,travel,index_right,index,cluster_n,cluster_type,cbsa
634901,31000US49460,9zdezezw,POINT (-97.39431 42.86839),0,0,0,0,2,6,6,6,shop,31000US49460
634910,31000US49460,9zdezsm2,POINT (-97.39740 42.87011),0,0,0,0,2,6,6,6,shop,31000US49460
634912,31000US49460,9zdezsne,POINT (-97.39569 42.86925),0,0,1,0,1,6,6,6,shop,31000US49460
634913,31000US49460,9zdezsny,POINT (-97.39534 42.86977),0,0,0,0,1,6,6,6,shop,31000US49460
634914,31000US49460,9zdezsnz,POINT (-97.39534 42.86994),0,0,0,1,2,6,6,6,shop,31000US49460
...,...,...,...,...,...,...,...,...,...,...,...,...,...
635087,31000US49460,9zdsp86u,POINT (-97.40084 42.89277),0,0,0,0,1,9,9,9,shop,31000US49460
635088,31000US49460,9zdsp8e1,POINT (-97.40049 42.89363),0,0,1,0,0,9,9,9,shop,31000US49460
635091,31000US49460,9zdsp8sx,POINT (-97.39843 42.89466),0,1,0,0,1,13,13,13,casual,31000US49460
635092,31000US49460,9zdsp8th,POINT (-97.39775 42.89414),0,0,0,0,2,13,13,13,casual,31000US49460
